In [1]:
%matplotlib inline
# %load_ext autoreload
# %autoreload 1 # Always reload before execution only modules imported with '%aimport'
# %aimport pipeline
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import random
import os
import sys
import itertools
sys.path.append('src/')
import nn
import process_data
import nibabel as nib
from math import floor, ceil

from sklearn.metrics import confusion_matrix
import scipy.sparse
from scipy.misc import imrotate, imresize
from scipy.ndimage.filters import gaussian_filter
from scipy.ndimage import rotate
from skimage import exposure
from skimage.io import imread, imsave
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from tensorflow.python.client import device_lib
local_device_protos = device_lib.list_local_devices()
print(local_device_protos)

import pipeline
import Unet

/home/jessica/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13539763196598832243
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 11698752717
locality {
  bus_id: 1
}
incarnation: 4026854566873107477
physical_device_desc: "device: 0, name: TITAN Xp, pci bus id: 0000:02:00.0"
]


In [2]:
def tensorboard():
    import webbrowser
    webbrowser.open('http://localhost:6006')

    !tensorboard --logdir="logs"

In [3]:
models_dir = '/media/jessica/Storage/models/u-net_v1-0'
model_name = 'new_train_test'

In [4]:
training_data_dir = "/home/jessica/Documents/hart-seg-ml/training_data"

In [5]:
raw_data_lst, seg_data_lst = pipeline.load_all_data(training_data_dir)

In [6]:
x_train, x_val, x_test, y_train, y_val, y_test = pipeline.split_data(raw_data_lst, seg_data_lst, 60, 10, 30)

In [7]:
mean = 0
weight_decay = 1e-6
learning_rate = 1e-4

In [16]:
import importlib
tf.summary.FileWriterCache.clear()
importlib.reload(Unet)
importlib.reload(nn)
tf.reset_default_graph()
sess = tf.Session()
model = Unet.Unet(mean, weight_decay, learning_rate, dropout=0.5)
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(max_to_keep=1, keep_checkpoint_every_n_hours=1)

(?, 512, 512, 64)


NameError: name 'x_train' is not defined

In [9]:
nn.train(sess, model.merged_summary_op, model, saver, x_train, y_train, x_val, y_val, x_test, y_test, epochs = 100, batch_size = 1, models_dir=models_dir, model_name=model_name, summary_writer=1)

1


InvalidArgumentError: Shape [-1,512,512,1] has negative dimensions
	 [[Node: x_test = Placeholder[dtype=DT_FLOAT, shape=[?,512,512,1], _device="/job:localhost/replica:0/task:0/gpu:0"]()]]

Caused by op 'x_test', defined at:
  File "/home/jessica/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/jessica/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/jessica/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/jessica/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/jessica/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 478, in start
    self.io_loop.start()
  File "/home/jessica/anaconda3/lib/python3.6/site-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  File "/ho

In [ ]:
print(nn.validate(sess, model, x_test, y_test))

In [ ]:
pipeline.save_model(models_dir, model_name, saver, sess)